# Exp Q1a: Finite-Size Scaling (width=0.5×)

## 目的
有限サイズスケーリングの検証：小さいモデル（width=0.5×）

## 実験設計
- **Model**: ResNet18 (width=0.5×)
- **η**: 0.4, 0.8
- **λ (η=0.4)**: [0.42, 0.44, 0.46, 0.48, 0.50]
- **λ (η=0.8)**: [0.38, 0.40, 0.42, 0.44, 0.46]
- **Seeds**: 10
- **Total**: 1 × 2 × 5 × 10 = **100 runs**

## 推定時間
~100 runs × 9 min ≈ **15 時間**

## 関連ノートブック
- Q1a (this): width=0.5×
- Q1b: width=1.0×
- Q1c: width=2.0×

In [ ]:
# ===== セットアップ =====
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
from datetime import datetime

# 共通の保存ディレクトリ（Q1a/b/cで共有）
EXP_NAME = 'exp_Q1_finite_size_scaling'
BASE_DIR = '/content/drive/MyDrive/dual-gradient-learning/Paper-A'

existing = glob.glob(f'{BASE_DIR}/{EXP_NAME}_*')
if existing:
    SAVE_DIR = sorted(existing)[-1]
    print(f'🔄 Using existing directory: {SAVE_DIR}')
else:
    TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    SAVE_DIR = f'{BASE_DIR}/{EXP_NAME}_{TIMESTAMP}'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'🆕 New experiment: {SAVE_DIR}')

os.makedirs(f'{SAVE_DIR}/figures', exist_ok=True)
print(f'Save directory: {SAVE_DIR}')
print(f'This notebook: Q1a (width=0.5×)')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils import parameters_to_vector
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')

In [ ]:
# ===== Width-Scalable ResNet =====

class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, in_planes, planes, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNetScalable(nn.Module):
    def __init__(self, width_mult=1.0, num_classes=10):
        super().__init__()
        self.width_mult = width_mult
        self.in_planes = int(64 * width_mult)
        
        c1 = int(64 * width_mult)
        c2 = int(128 * width_mult)
        c3 = int(256 * width_mult)
        c4 = int(512 * width_mult)
        
        self.conv1 = nn.Conv2d(3, c1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(c1)
        
        self.layer1 = self._make_layer(c1, 2, stride=1)
        self.layer2 = self._make_layer(c2, 2, stride=2)
        self.layer3 = self._make_layer(c3, 2, stride=2)
        self.layer4 = self._make_layer(c4, 2, stride=2)
        
        self.linear = nn.Linear(c4 * BasicBlock.expansion, num_classes)
    
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes * BasicBlock.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# このノートブックのwidth
WIDTH_MULT = 0.5
m = ResNetScalable(width_mult=WIDTH_MULT)
params = sum(p.numel() for p in m.parameters())
print(f'ResNet18 (width={WIDTH_MULT}×): {params:,} parameters')
del m

In [ ]:
# ===== IndexedDataset =====
class IndexedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    def __getitem__(self, idx):
        img, label = self.dataset[idx]
        return img, label, idx
    def __len__(self):
        return len(self.dataset)

In [ ]:
# ===== 実験パラメータ =====
BATCH_SIZE = 256
NUM_WORKERS = 4
EPOCHS = 100
LR = 0.1
K = 16

# このノートブック専用
WIDTH_MULT = 0.5

NOISE_LAMBDA_MAP = {
    0.4: [0.42, 0.44, 0.46, 0.48, 0.50],
    0.8: [0.38, 0.40, 0.42, 0.44, 0.46]
}

SEEDS = list(range(10))

experiments = []
for eta, lambdas in NOISE_LAMBDA_MAP.items():
    for lam in lambdas:
        for seed in SEEDS:
            experiments.append({
                'width_mult': WIDTH_MULT,
                'noise_rate': eta,
                'lambda': lam,
                'seed': seed
            })

print(f'Width multiplier: {WIDTH_MULT}×')
print(f'Noise rates: {list(NOISE_LAMBDA_MAP.keys())}')
print(f'Total experiments: {len(experiments)}')
print(f'Estimated time: {len(experiments) * 9 / 60:.1f} hours')

# Save config
config = {
    'experiment': 'exp_Q1a_finite_size_scaling',
    'description': f'Finite-size scaling: width={WIDTH_MULT}×',
    'parameters': {
        'width_mult': WIDTH_MULT,
        'noise_lambda_map': NOISE_LAMBDA_MAP,
        'seeds': SEEDS,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'lr': LR,
        'K': K
    },
    'total_runs': len(experiments)
}
with open(f'{SAVE_DIR}/exp_Q1a_config.json', 'w') as f:
    json.dump(config, f, indent=2)

In [ ]:
# ===== ユーティリティ関数 =====
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

def load_cifar10():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    return trainset, testset

def inject_label_noise(labels, noise_rate, seed):
    np.random.seed(seed)
    labels = np.array(labels)
    n_samples = len(labels)
    n_noisy = int(noise_rate * n_samples)
    noisy_idx = np.random.choice(n_samples, n_noisy, replace=False)
    noisy_labels = labels.copy()
    for idx in noisy_idx:
        choices = [l for l in range(10) if l != labels[idx]]
        noisy_labels[idx] = np.random.choice(choices)
    return noisy_labels

def get_data_loaders(trainset, testset):
    train_loader = DataLoader(IndexedDataset(trainset), batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
    return train_loader, test_loader

def evaluate(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

In [ ]:
# ===== Dual-Gradient Learning =====
def train_dual_gradient(model, train_loader, test_loader, clean_labels, noisy_labels, lam):
    optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    clean_labels_tensor = torch.tensor(clean_labels, device=device)
    noisy_labels_tensor = torch.tensor(noisy_labels, device=device)
    
    cached_g_value = None
    global_step = 0
    
    best_acc = 0
    cos_history = []
    history = {'epoch': [], 'test_acc': [], 'test_error': []}
    
    for epoch in range(EPOCHS):
        model.train()
        epoch_cos = []
        
        for inputs, _, indices in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            indices = indices.to(device, non_blocking=True)
            batch_noisy = noisy_labels_tensor[indices]
            batch_clean = clean_labels_tensor[indices]
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss_struct = criterion(outputs, batch_noisy)
            loss_struct.backward(retain_graph=True)
            g_struct = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            if global_step % K == 0 or cached_g_value is None:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss_value = criterion(outputs, batch_clean)
                loss_value.backward()
                cached_g_value = parameters_to_vector([p.grad for p in model.parameters()]).clone()
            
            g_struct_norm = g_struct / (g_struct.norm() + 1e-12)
            g_value_norm = cached_g_value / (cached_g_value.norm() + 1e-12)
            
            cos_sim = (g_struct_norm @ g_value_norm).item()
            epoch_cos.append(cos_sim)
            
            g_mix = (1 - lam) * g_struct_norm + lam * g_value_norm
            
            optimizer.zero_grad()
            idx = 0
            for p in model.parameters():
                numel = p.numel()
                p.grad = g_mix[idx:idx+numel].view(p.shape).clone()
                idx += numel
            optimizer.step()
            global_step += 1
        
        scheduler.step()
        cos_history.append(np.mean(epoch_cos))
        
        if (epoch + 1) % 10 == 0:
            acc = evaluate(model, test_loader)
            best_acc = max(best_acc, acc)
            history['epoch'].append(epoch + 1)
            history['test_acc'].append(acc)
            history['test_error'].append(1 - acc)
    
    final_acc = evaluate(model, test_loader)
    avg_cos = np.mean(cos_history)
    
    return final_acc, max(best_acc, final_acc), avg_cos, history

In [ ]:
# ===== データ準備 =====
trainset, testset = load_cifar10()
clean_labels = np.array(trainset.targets)
train_loader, test_loader = get_data_loaders(trainset, testset)

print('Data prepared')

warmup_model = ResNetScalable(width_mult=WIDTH_MULT).to(device)
for _ in range(20):
    _ = warmup_model(torch.randn(BATCH_SIZE, 3, 32, 32, device=device))
del warmup_model
torch.cuda.empty_cache()
print('GPU warmed up')

In [ ]:
# ===== メイン実験ループ =====
results = []
checkpoint_file = f'{SAVE_DIR}/exp_Q1a_checkpoint.json'
completed = set()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        results = json.load(f)
    for r in results:
        completed.add((r['width_mult'], r['noise_rate'], r['lambda'], r['seed']))
    print(f'Checkpoint loaded: {len(results)} runs completed')

total = len(experiments)

for exp in experiments:
    width = exp['width_mult']
    eta = exp['noise_rate']
    lam = exp['lambda']
    seed = exp['seed']
    
    if (width, eta, lam, seed) in completed:
        continue
    
    run_num = len(completed) + 1
    print(f'\n[{run_num}/{total}] width={width}× η={eta} λ={lam} seed={seed}')
    
    set_seed(seed)
    noisy_labels = inject_label_noise(clean_labels, eta, seed)
    
    model = ResNetScalable(width_mult=width).to(device)
    
    start_time = time.time()
    final_acc, best_acc, avg_cos, history = train_dual_gradient(
        model, train_loader, test_loader, clean_labels, noisy_labels, lam
    )
    elapsed = time.time() - start_time
    
    result = {
        'experiment_id': f'Q1a-{run_num:03d}',
        'experiment': 'exp_Q1a_finite_size_scaling',
        'width_mult': width,
        'noise_rate': eta,
        'lambda': lam,
        'seed': seed,
        'test_acc': final_acc,
        'test_error': 1 - final_acc,
        'best_acc': best_acc,
        'best_error': 1 - best_acc,
        'avg_cos_struct_value': avg_cos,
        'time_seconds': elapsed,
        'history': history
    }
    results.append(result)
    completed.add((width, eta, lam, seed))
    
    status = '✅' if best_acc > 0.85 else ('⚠️ COLLAPSE' if best_acc < 0.5 else '')
    print(f'  Error: {1-best_acc:.4f} | cos: {avg_cos:.4f} | Time: {elapsed/60:.1f} min {status}')
    
    with open(checkpoint_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    remaining = total - run_num
    eta_hours = remaining * elapsed / 3600
    print(f'  Progress: {run_num}/{total} | ETA: {eta_hours:.1f} hours')
    
    del model
    torch.cuda.empty_cache()

print('\n' + '='*70)
print('Q1a (width=0.5×) COMPLETED!')
print('='*70)

In [ ]:
# ===== 結果保存 =====
import pandas as pd

with open(f'{SAVE_DIR}/exp_Q1a_results.json', 'w') as f:
    json.dump(results, f, indent=2)

results_flat = [{k: v for k, v in r.items() if k != 'history'} for r in results]
df = pd.DataFrame(results_flat)
df.to_csv(f'{SAVE_DIR}/exp_Q1a_results.csv', index=False)

print(f'Results saved to {SAVE_DIR}')
print(f'Next: Run Q1b (width=1.0×) and Q1c (width=2.0×)')